In [1]:
import dask.dataframe as dd
import dask
import numpy as np
from dask import delayed
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import cv2
from glob import glob
import json
import dask.array as da
import time
import xgboost as xgb
import pandas as pd
import os

@delayed
def padding(data, array_len, col_len):
    pad = np.zeros((array_len, col_len))
    length = min(array_len, len(data))
    pad[:length] = data[:length]
    return pad

path = os.getcwd()
csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
# print(cluster,client)
# client.close()

# http://localhost:8787/status

distributed.diskutils - INFO - Found stale lock file and directory '/home/gob/repos/Dacon_tabular_img_classification_model/dask-worker-space/worker-98rgb0nf', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/gob/repos/Dacon_tabular_img_classification_model/dask-worker-space/worker-wrmtbyxd', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/gob/repos/Dacon_tabular_img_classification_model/dask-worker-space/worker-rfuxhymv', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/gob/repos/Dacon_tabular_img_classification_model/dask-worker-space/worker-lsyisfe2', purging


In [2]:
#making_label_set
# 
crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
risk = {'1':'초기','2':'중기','3':'말기'}


#ensemble_labler
label_description = {}
for key, value in disease.items():
    label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
    for disease_code in value:
        for risk_code in risk:
            label = f'{key}_{disease_code}_{risk_code}'
            label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'

global ensemble_label_encoder
ensemble_label_encoder = {key:idx for idx, key in enumerate(label_description)}
ensemble_label_decoder = {val:key for key, val in ensemble_label_encoder.items()}

#crop_labler
crop_label_description = {}
for key, value in disease.items():
    crop_label_description[f'{key}'] = f'{crop[key]}'
    
global crop_label_encoder
crop_label_encoder = {key:idx for idx, key in enumerate(crop_label_description)}
crop_label_decoder = {val:key for key, val in crop_label_encoder.items()}


#disease_labler
disease_label_description = {}
for key, value in disease.items():
    disease_label_description[f'00'] = f'{crop[key]}'
    for disease_code in value:
        label = f'{disease_code}'
        disease_label_description[label] = f'{disease_code}'

global disease_label_encoder
disease_label_encoder = {key:idx for idx, key in enumerate(disease_label_description)}
disease_label_decoder = {val:key for key, val in disease_label_encoder.items()}


#risk_labler
risk_label_description = {}
for key, value in risk.items():
    label = f'{key}'
    risk_label_description[label] = f'{key}'

global risk_label_encoder
risk_label_encoder = {key:idx for idx, key in enumerate(risk_label_description)}
risk_label_decoder = {val:key for key, val in risk_label_encoder.items()}





In [3]:
@delayed
def label_encoding(label):
    global label_encoder
    encoded_label = ensemble_label_encoder[label]
    return encoded_label

def getlable(jsonpath):
    with open(jsonpath, 'r') as f:
        json_file = json.load(f)

    crop = json_file['annotations']['crop']
    disease = json_file['annotations']['disease']
    risk = json_file['annotations']['risk']
    label = f'{crop}_{disease}_{risk}'
    return label
    
def get_label_list(labelpath_list):
    labelarr = np.array([])
    # labelarr = da.array([])
    for ind,json_path in enumerate(labelpath_list):
        # label = label_encoding(getlable(json_path))
        # label = da.array(np.array(label_encoding(getlable(json_path))))
        label = np.array(label_encoding(getlable(json_path)))
        labelarr = np.append(labelarr,label)
    return labelarr

데이터 준비

In [4]:
row_csv = dd.read_csv("/mnt/c/data/dacon_parm/train/10027/*.csv",dtype={'외부 누적일사 평균': 'object','내부 이슬점 최고': 'object',
       '내부 이슬점 최저': 'object',
       '내부 이슬점 평균': 'object'})

In [5]:
#making_csv_set
row_csv = dd.read_csv("/mnt/c/data/dacon_parm/train/*.csv",dtype={'외부 누적일사 평균': 'object','내부 이슬점 최고': 'object',
       '내부 이슬점 최저': 'object',
       '내부 이슬점 평균': 'object'})
partitions = row_csv.to_delayed()
# datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features].values,290,9) for part in partitions]
datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features][part != "-"].dropna().values,290,9) for part in partitions[:]]
X_train = np.array(dask.compute(*datas))


IndexError: list index out of range

In [5]:
label_list = sorted(glob(path + "\\data_1\\train\\*\\*.json"))
y_train = get_label_list(label_list)
# y_train = y_train.rechunk(5767)
results = dask.compute(*y_train)
label = np.array(results)
# print(label.shape)

In [6]:
label.shape, X_train.shape

((5767,), (5767, 290, 9))

In [7]:
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

for i in range(len(csv_features)):
  start = time.time()
  globals()["XGB_{}".format(i)] = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X_train[:,:,i],label)
  print(time.time()-start)

C:\Users\rhqud\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py:499: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(
C:\Users\rhqud\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


5.571685552597046
6.348506450653076
8.041103601455688
9.475648164749146
9.165439367294312
9.312985181808472
9.130616903305054
9.028754234313965
7.739068508148193


모델학습

In [50]:
def process_x(dataset):
    global XGB_0, XGB_1, XGB_2, XGB_3, XGB_4, XGB_5, XGB_6, XGB_7, XGB_8 
    
    x0=XGB_0.predict(dataset[:,:,0])
    x1=XGB_1.predict(dataset[:,:,1])
    x2=XGB_2.predict(dataset[:,:,2])
    x3=XGB_3.predict(dataset[:,:,3])
    x4=XGB_4.predict(dataset[:,:,4])
    x5=XGB_5.predict(dataset[:,:,5])
    x6=XGB_6.predict(dataset[:,:,6])
    x7=XGB_7.predict(dataset[:,:,7])
    x8=XGB_8.predict(dataset[:,:,8])
    arr_x = np.array([x0,x1,x2,x3,x4,x5,x6,x7,x8,]).T
    return arr_x
    

In [52]:
x0=XGB_0.predict(X_train[:,:,0])
x1=XGB_1.predict(X_train[:,:,1])
x2=XGB_2.predict(X_train[:,:,2])
x3=XGB_3.predict(X_train[:,:,3])
x4=XGB_4.predict(X_train[:,:,4])
x5=XGB_5.predict(X_train[:,:,5])
x6=XGB_6.predict(X_train[:,:,6])
x7=XGB_7.predict(X_train[:,:,7])
x8=XGB_8.predict(X_train[:,:,8])
processed_x = np.array([x0,x1,x2,x3,x4,x5,x6,x7,x8,]).T

In [53]:
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

start = time.time()
XGB_G = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(processed_x,label)
print(time.time()-start)

C:\Users\rhqud\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py:499: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(
C:\Users\rhqud\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.5968475341796875


평가

In [54]:
def csv_path(label):
    # csv_path = f'{file}\\{file_name}.csv'
    # l_path = os.getcwd() + '\\data\\test\\%s\\%s.csv'%{label,label}
    csv_path = os.getcwd() +f'\\data_1\\test\\{label}\\{label}.csv'
    return csv_path
    
def img_path(label):
    # csv_path = f'{file}\\{file_name}.csv'
    # l_path = os.getcwd() + '\\data\\test\\%s\\%s.csv'%{label,label}
    img_path = os.getcwd() +f'\\data_1\\test\\{label}\\{label}.jpg'
    return img_path

In [55]:
#making_csv_set
test_csv = dd.read_csv(path + "\\data_1\\test\\*\\*csv",dtype={'외부 누적일사 평균': 'object','내부 이슬점 최고': 'object',
    '내부 이슬점 최저': 'object',
    '내부 이슬점 평균': 'object'})
partitions = test_csv.to_delayed()
datas_test = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features][part != "-"].dropna().values,290,9) for part in partitions[:]]
# X_train = np.array(dask.compute(*datas_test))

In [70]:
temp = np.array(dask.compute(*datas_test[:1000]))


In [ ]:
XGB_G.predict(process_x(np.array(dask.compute(*datas_test[:1000]))))

In [80]:
submission = pd.read_csv(path + '\\data_1\\sample_submission.csv')
# submission['label'] = preds
minibatch_size = 500
data_length = len(submission)
loop_num = int(np.floor(data_length/minibatch_size))

# for i 
# xgb.fit(x_prime_train,y_train,xgb_model = xgb)
count = 0
for i in range(loop_num):
# for i in range(2):
    print(i)
    if i !=loop_num-1:
        
        start_p = count*minibatch_size
        end_p = (count+1)*minibatch_size
        
        #예측
        pred_raw = XGB_G.predict(process_x(np.array(dask.compute(*datas_test[start_p:end_p]))))
        submission.loc[start_p:end_p-1,'label'] = np.array([ensemble_label_decoder[int(val)] for val in pred_raw])
        
    else:
        start_p = count*minibatch_size
       
        #예측
        pred_raw = XGB_G.predict(process_x(np.array(dask.compute(*datas_test[start_p:]))))
        submission.loc[start_p:,'label'] = np.array([ensemble_label_decoder[int(val)] for val in pred_raw])
    
    count += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102


In [ ]:
submission.to_csv('submission\\gogo_submission_03.csv', index=False)